Import Keras

In [ ]:
library(keras)

Create the Data Generators

In [ ]:
train_data_gen <- image_data_generator(
      preprocessing_function = nasnet_preprocess_input, #split into 80% training and 20% validation data
      rotation_range = 40, #Data Augmentation
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      shear_range = 0.2,
      zoom_range = 0.2,
      horizontal_flip = TRUE,
      fill_mode = "nearest"
)

# Validation data shouldn't be augmented! But it should also be scaled.
valid_data_gen <- image_data_generator(
      preprocessing_function = nasnet_preprocess_input
)
   

Specify the Params

In [ ]:
img_width <- 331L
img_height <- 331L
channels <- 3L #RGB has 3 colours

batch_size <- 32L

num_classes <- 3L

train_image_files_path <- '../input/testing/Train'

Load the Data and convert IMG to arrays

In [ ]:
# training images
train_it <- flow_images_from_directory(train_image_files_path, 
                                       train_data_gen,
                                       target_size = c(img_width, img_height),
                                       class_mode = "categorical",
                                       batch_size = batch_size,
                                       shuffle = TRUE) 

# validation images
val_it <- flow_images_from_directory(train_image_files_path, 
                                     valid_data_gen,
                                     target_size = c(img_width, img_height),
                                     class_mode = "categorical",
                                     batch_size = batch_size,
                                     shuffle = FALSE)

Check if loading worked

In [ ]:
str(reticulate::iter_next(train_it))
str(reticulate::iter_next(val_it))

Create the Model

In [ ]:
# Load the pretrained Model (Transfer Learning)
base_model <- application_nasnetlarge(weights = 'imagenet', include_top = FALSE, 
                                input_shape = c(img_width, img_height, channels)) # We stripped off the input layer and the classifier layer

# Examine the Architecture
summary(base_model)

Fine Tuning

In [ ]:
# define new classifier layers
predictions <- base_model$output %>%
  layer_global_average_pooling_2d(trainable = T) %>%
  layer_dense(num_classes, trainable=T) %>%   
  layer_activation("softmax", trainable=T) # Softmax is used for Multi-label classification

# add the layers to the model
model <- keras_model(inputs = base_model$input, outputs = predictions)
freeze_weights(base_model) # Freeze the weights of the base_model as we only want to train the classifier

summary(model)

Compile the Model

In [ ]:
model %>% compile(
  optimizer = optimizer_sgd(lr = 0.001, momentum = 0.9, nesterov = TRUE),
  loss = 'categorical_crossentropy', # used in multi-label classification
  metrics = c('accuracy')
)

Start Training

In [ ]:
hist <- model %>% fit_generator(
  train_it,
  validation_data = val_it,
  epochs=10, 
  steps_per_epoch = 1,
  validation_steps = 1,
  verbose = 2
)

Save the Model

In [ ]:
save_model_hdf5(model, "model.h5")

Load Model

In [ ]:
model <- load_model_hdf5("model.h5")

Visualise the Training History

In [ ]:
plot(hist)

In [ ]:
z <- evaluate_generator(model, 
                        val_it, 
                        steps = 1)
z

Load the Test Image

In [ ]:
# load the image
img_path <- "../input/testing/Train/1/download1.jpeg"
img <- image_load(img_path, target_size = c(img_width,img_height))
x <- image_to_array(img)

Preprocess the IMG

In [ ]:
x <- array_reshape(x, c(1, dim(x)))
x <- nasnet_preprocess_input(x)
x

Prediction

In [ ]:
# make predictions then decode and print them
pred <- model %>% predict(x)

In [ ]:
pred

In [ ]:
# Implement a dictionary (Key-Value) for mapping prediction to labels

d <- list()

d[[1]] <- "Apples"

d[[2]] <- "Oranges"

d[[3]] <- "Bananas"

In [ ]:
d[[which.max(pred)]]

In [ ]:
# test_image_files_path <- '../input/testing-2/Test'

# test_data_gen <- image_data_generator(
#       preprocessing_function = nasnet_preprocess_input
# ) 


# test_it <- flow_images_from_directory(test_image_files_path, 
#                                      test_data_gen,
#                                      target_size = c(img_width, img_height),
#                                      class_mode = NULL,
#                                      batch_size = batch_size,
#                                      shuffle = FALSE)

# str(reticulate::iter_next(test_it))